In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('C:\\Users\\Admin\\Downloads\\creditcard.csv')

In [4]:
time = np.array(data['Time'])
delta_time = time[1:] - time[:-1]

In [5]:
data['Time'] = np.hstack((0, delta_time))

In [6]:
test_point = np.array(data)[0: 50][np.newaxis, :, :]

In [7]:
test_point.shape

(1, 50, 31)

In [7]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,0.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,1.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,1.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,1.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,1.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,0.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [8]:
def data_generator(data, sq_len, n_features):
    inputs = tf.zeros((len(data), 0, n_features))

    for i in range(sq_len):
        inputs = tf.concat((inputs, np.array(data.shift(-i))[:,np.newaxis,:]), axis = 1)

    return inputs[:-sq_len+1,:,:-1], inputs[:-sq_len+1, -1, -1]

In [9]:
inputs, labels = data_generator(data, 50, 31)

In [10]:
inputs.shape

TensorShape([284758, 50, 30])

In [11]:
labels.shape

TensorShape([284758])

In [13]:
class Transactional_Extension(tf.keras.layers.Layer):
    def __init__(self, per_step: int, k: int):
        super(Transactional_Extension, self).__init__()
        self.per_step = per_step
        self.k = k
    
    def call(self, input):
        pad = tf.tile(input[:, 0:1], [1, self.per_step - 1, 1])
        fix_data = tf.concat((pad, input), axis = 1)
        data_split = tf.convert_to_tensor([fix_data[:, i : i + self.per_step] for i in range(input.shape[1])])
        data_cells = tf.transpose(tf.convert_to_tensor([data_split[i : i + self.k] for i in range(data_split.shape[0] - self.k + 1)]), perm=[2, 0, 1, 3, 4])
        return data_cells

In [107]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, k, w_aq, w_ah, b_a, v):
        self.w_aq = w_aq
        self.w_ah = w_ah
        self.b_a = b_a
        self.k = k
        self.v = v
        super(Attention, self).__init__()

    def call(self, ht, ct, hi):
        q = ht[:,tf.newaxis,...] + ct[:,tf.newaxis,...]
        aq = self.w_aq(q)
        ah = self.w_ah(hi[:, -self.k:])
        oti = tf.math.tanh(aq + ah + self.b_a)
        alpha = self.v(oti)
        alpha_exp = tf.math.exp(alpha)
        alphati = alpha_exp / tf.math.reduce_sum(alpha_exp)
        e = tf.reduce_sum(tf.squeeze(tf.reduce_sum(alphati, axis=1)) * hi[:, -self.k:], axis=1)
        e_exp = tf.math.exp(e)
        return e_exp / tf.reduce_sum(e_exp)

In [108]:
class InteractionModule(tf.keras.layers.Layer):
    def __init__(self, wh, we, wg, b_h):
        self.wh = wh
        self.we = we
        self.wg = wg
        self.b_h = b_h
        super(InteractionModule, self).__init__()

    def call(self, ht, e, g):
        _h = self.wh(ht)
        _e = self.we(e)
        _g = self.wg(g)
        new_h = tf.math.tanh(_h + _e + _g + self.b_h)
        return new_h

In [109]:
class cell(tf.keras.layers.Layer):
    def __init__(self, w_sh, w_sx, w_st, b_s, 
                 w_fh, w_fx, w_fs, b_f, 
                 w_ih, w_ix, w_is, b_i, 
                 w_Th, w_Tx, w_Ts, b_T, 
                 w_elph, w_elpx, w_elps, b_elp, 
                 w_oh, w_ox, w_os, b_o, 
                 w_aq, w_ah, b_a, 
                 k, v, wh, we, wg, b_h):
        super(cell, self).__init__()
        self.w_sh = w_sh
        self.w_sx = w_sx
        self.w_st = w_st
        self.b_s = b_s

        self.w_fh = w_fh
        self.w_fx = w_fx
        self.w_fs = w_fs
        self.b_f = b_f

        self.w_ih = w_ih
        self.w_ix = w_ix
        self.w_is = w_is
        self.b_i = b_i

        self.w_Th = w_Th
        self.w_Tx = w_Tx
        self.w_Ts = w_Ts
        self.b_T = b_T

        self.w_elph = w_elph
        self.w_elpx = w_elpx
        self.w_elps = w_elps
        self.b_elp = b_elp

        self.w_oh = w_oh
        self.w_ox = w_ox
        self.w_os = w_os
        self.b_o = b_o

        self.attn = Attention(k, w_aq, w_ah, b_a, v)
        self.IM = InteractionModule(wh, we, wg, b_h)

    def call(self, x_t, prev_h, prev_c, dt, store):
        s_x = self.w_sx(x_t)
        s_t = self.w_st(dt)
        s_h = self.w_sh(prev_h)

        s = tf.math.tanh(s_h + s_x + s_t[..., tf.newaxis] + self.b_s)

        f_x = self.w_fx(x_t)
        f_t = self.w_fs(s)
        f_h = self.w_fh(prev_h)

        f = tf.math.sigmoid(f_h + f_x + f_t + self.b_f)

        i_x = self.w_ix(x_t)
        i_t = self.w_is(s)
        i_h = self.w_ih(prev_h)

        i = tf.math.sigmoid(i_h + i_x + i_t + self.b_i)

        T_x = self.w_Tx(x_t)
        T_t = self.w_Ts(s)
        T_h = self.w_Th(prev_h)

        T = tf.math.sigmoid(T_h + T_x + T_t + self.b_T)

        elp_x = self.w_elpx(x_t)
        elp_t = self.w_elps(s)
        elp_h = self.w_elph(prev_h)

        elp = tf.math.tanh(elp_h + elp_x + elp_t + self.b_elp)

        new_c = f * prev_c + i * elp + T * s

        o_x = self.w_ox(x_t)
        o_t = self.w_os(s)
        o_h = self.w_oh(prev_h)

        o = tf.math.sigmoid(o_h + o_x + o_t + self.b_o)

        h = o * tf.math.sigmoid(new_c)

        attn_score = self.attn(h, new_c, store)
        print(attn_score.shape)
        new_h = self.IM(h, attn_score, x_t)
        return new_c, new_h


In [110]:
class TH_LSTM(tf.keras.layers.Layer):
    def __init__(self, len: int, per_step: int, k: int):
        super(TH_LSTM, self).__init__()
        self.units = len - k + 1
        self.k = k
        self.per_step = per_step
        self.TE = Transactional_Extension(per_step = per_step, k = k)
        self.norm = tf.keras.layers.LayerNormalization()

    def build(self, input_shape):
        batch = input_shape[0]
        input_shape = [self.TE.k, self.per_step, input_shape[-1]]
        self.w_sh = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Time_aware_input_hidden_state')
        self.w_sx = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Time_aware_input')
        self.w_st = tf.keras.layers.EinsumDense('...xf,xd->...xd', output_shape=input_shape[:-1], bias_axes='xd', name='Time_aware_time_step')
        self.b_s = self.add_weight('Time_aware_bias', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.w_fh = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Forget_gate_hidden_state')
        self.w_fx = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Forget_gate_input')
        self.w_fs = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Forget_gate_time_stamp')
        self.b_f = self.add_weight('Forget_gate_bias', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.w_ih = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Input_gate_hidden_state')
        self.w_ix = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Input_gate_input')
        self.w_is = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Input_gate_time_stamp')
        self.b_i = self.add_weight('Input_gate_bias', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.w_Th = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Time-aware_gate_hidden_state')
        self.w_Tx = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Time-aware_gate_input')
        self.w_Ts = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Time-aware_gate_time_stamp')
        self.b_T = self.add_weight('Time-aware_gate_bias', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.w_elph = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Generated_candidate_cell_state_hidden_state')
        self.w_elpx = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Generated_candidate_cell_state_input')
        self.w_elps = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Generated_candidate_cell_state_time_stamp')
        self.b_elp = self.add_weight('Generated_candidate_cell_state_bias', shape = input_shape, initializer='random_normal', trainable=True)

        self.w_oh = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Output_gate_hidden_state')
        self.w_ox = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Output_gate_input')
        self.w_os = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='Output_gate_time_stamp')
        self.b_o = self.add_weight('Output_gate_bias', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.w_aq = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='aq')
        self.w_ah = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='ah')
        self.b_a = self.add_weight('ba', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.w_h = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='wh')
        self.w_e = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='we')
        self.w_g = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='wg')
        self.b_h = self.add_weight('ba', shape = input_shape[1:], initializer='random_normal', trainable=True)

        self.v = tf.keras.layers.EinsumDense('...gxf,gxd->...gxd', output_shape=input_shape, bias_axes='xd', name='v')

        self.cells = [cell(self.w_sh, self.w_sx, self.w_st, self.b_s, 
                           self.w_fh, self.w_fx, self.w_fs, self.b_f, 
                           self.w_ih, self.w_ix, self.w_is, self.b_i, 
                           self.w_Th, self.w_Tx, self.w_Ts, self.b_T, 
                           self.w_elph, self.w_elpx, self.w_elps, self.b_elp, 
                           self.w_oh, self.w_ox, self.w_os, self.b_o, 
                           self.w_aq, self.w_ah, self.b_a, 
                           self.k, self.v, self.w_h, self.w_e, self.w_g, self.b_h) 
                      for i in range(self.units)]
        
        self.h_init = self.add_weight('h_init', shape=[1] + input_shape, initializer='zeros', trainable=False)
        self.c_init = self.add_weight('h_init', shape=[1] + input_shape, initializer='zeros', trainable=False)
        return super().build(input_shape)
    
    def call(self, data_sq):
        shape = tf.shape(data_sq)
        str_mem = tf.zeros((shape[0], self.k, self.k, self.per_step, data_sq.shape[-1]))
        print(str_mem)
        data_split = self.TE(data_sq)
        h = self.h_init
        c = self.c_init
        
        for i in range(self.units):
            c, h = self.cells[i](data_split[:, i,..., 1:], h, c, data_split[:, i,... , 0], str_mem)
            str_mem = tf.concat((str_mem, h[:, tf.newaxis, ...]), axis = 1)
        h = self.norm(h)
        c = self.norm(c)
        return h

In [111]:
TH = TH_LSTM(50, 20, 10)

In [112]:
TH(test_point[0:1])

tf.Tensor(
[[[[[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]]

   [[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]]

   [[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]]

   ...

   [[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]]

   [[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]]

   [[0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    [0. 0. 0. ... 0. 0. 0.]
    ...

<tf.Tensor: shape=(1, 10, 20, 31), dtype=float32, numpy=
array([[[[-0.97570467,  1.176692  ,  1.0733176 , ...,  1.1885358 ,
          -0.9122394 ,  1.1905085 ],
         [-1.0242919 ,  1.0716457 ,  1.1013155 , ..., -0.7113095 ,
           0.38469014, -0.09864672],
         [ 0.93249315, -1.0767227 ,  1.1403899 , ...,  0.35825592,
          -1.145021  , -0.71911633],
         ...,
         [ 0.6110509 ,  1.4875286 , -1.0817502 , ...,  1.0255133 ,
           1.4889672 , -0.8318598 ],
         [ 1.2933362 ,  0.39893702, -0.56880236, ..., -1.5941602 ,
           0.31802157,  0.47317508],
         [-1.0979834 , -1.0902103 , -1.0932243 , ..., -1.095734  ,
          -0.32377937, -1.1039878 ]],

        [[-1.3263509 ,  0.578947  ,  0.5166446 , ..., -0.7971894 ,
           0.6361059 , -0.673308  ],
         [ 1.014041  ,  1.037879  ,  1.0133493 , ...,  0.7067518 ,
          -1.2833288 ,  1.0228721 ],
         [ 0.80001724, -0.8055412 ,  0.06684397, ...,  0.12931502,
           0.01220575,  1.32

In [18]:
model = tf.keras.Sequential()
model.add(TH_LSTM(50, 20, 10))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [20]:
list(inputs.shape[1:])

[50, 30]

In [21]:
model.build(input_shape=[None] + list(inputs.shape[1:]))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 th_lstm (TH_LSTM)           (None, 10, 20, 30)        174460    
                                                                 
 flatten (Flatten)           (None, 6000)              0         
                                                                 
 dense (Dense)               (None, 1)                 6001      
                                                                 
Total params: 180461 (704.93 KB)
Trainable params: 168461 (658.05 KB)
Non-trainable params: 12000 (46.88 KB)
_________________________________________________________________


In [23]:
labels

<tf.Tensor: shape=(284758,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>

In [24]:
model.fit(x=inputs, y=labels, batch_size=64, epochs=100, shuffle=False, validation_split=0.2)

Epoch 1/100
